In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
df = load_dataset("Mubeen161/DEVOPS")

Generating train split: 100%|██████████| 42819/42819 [00:00<00:00, 487351.64 examples/s]


In [6]:
import json
data_dict = df['train'].to_dict()
with open('devops-dataset.json', 'w') as f:
        json.dump(data_dict, f, indent=2)

In [2]:
import json

with open('devops-42k.json', 'r') as file:
    data = json.load(file)

print(data.get("Prompt")[0])
print()
print(data.get("Instruction")[0])
print()
print(data.get("Response")[0])


Permanent browser cache using ServiceWorker


I am designing a JavaScript secure loader. The loader is inlined in the index.html. The goal of the secure loader is to only load JavaScript resources are trusted. The contents of index.html are mostly limited to the secure loader. For security purposes, I want index.html (as stored in cache) to never change, even if my website is hacked.
How can I cache index.html without the server being able to tamper with the cache? I am wondering if ServiceWorkers can help. Effectively, the index.html would register a service worker for fetching itself from an immutable cache (no network request is even made).










            4
        















                +100
            







in chrome you can use FileSystem API
http://www.noupe.com/design/html5-filesystem-api-create-files-store-locally-using-javascript-webkit.html this allows you to then save and read files from a sand-boxed file-system though the browser.
As for other support it

In [8]:
import json
import re
from html import unescape

class DatasetCleaner:
    def __init__(self):
        # Compile regex patterns for better performance
        self.patterns = {
            'html_tags': re.compile(r'<[^>]+>'),
            'stackoverflow_artifacts': re.compile(r'\b(Share|Improve this answer|Follow|edited|answered|Add a comment)\b.*', re.IGNORECASE),
            'user_reputation': re.compile(r',\s*\d{1,6}\s*(?:\d+\s*(?:silver|bronze|gold)\s*badges?)*'),
            'username_reputation': re.compile(r'\b[A-Za-z0-9_]+\b\s*\d{1,6}\s*(?:\d+\s*(?:silver|bronze|gold)\s*badges?)*'),
            'isolated_numbers': re.compile(r'^\s*\d+\s*$', re.MULTILINE),
            'number_patterns': re.compile(r'\s+\d{1,4}\s+'),
            'timestamps': re.compile(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},?\s+\d{4}(?:\s+at\s+\d{1,2}:\d{2})?'),
            'time_colons': re.compile(r',\s*\d{4}\s*:\d{2}'),
            'user_mentions': re.compile(r'–\s*[A-Za-z0-9_]+'),
            'retryable_errors': re.compile(r'\[!\]\s*retryable error.*', re.DOTALL),
            'duplicate_questions': re.compile(r'This question already has an answer here:.*', re.DOTALL),
            'answer_counts': re.compile(r'\(\d+\s*answer\)'),
            'time_ago': re.compile(r'\b\d+\s*(?:year|month|day|hour|minute)s?\s+ago\.?'),
            'duplicate_markers': re.compile(r'\[duplicate\]'),
            'common_artifacts': re.compile(r'\b(HIH|Hope it helps?)\b'),
            'revision_thanks': re.compile(r'Revision:\s*\(thanks.*?\)'),
            'multiple_newlines': re.compile(r'\n\s*\n\s*\n+'),
            'multiple_spaces': re.compile(r'[ \t]+'),
            'meaningless_lines': re.compile(r'^[\d\s\+\-\|\*\.,;:]+$'),
            'no_alphanumeric': re.compile(r'^[^\w]*$'),
            'meaningful_words': re.compile(r'\b[a-zA-Z]{3,}\b')
        }
    
    def clean_text(self, text):
        """
        Clean text by removing HTML tags, extra whitespace, and other artifacts
        """
        if not text:
            return ""
        
        # Convert to string if not already
        text = str(text)
        
        # Unescape HTML entities
        text = unescape(text)
        
        # Remove HTML tags
        text = self.patterns['html_tags'].sub('', text)
        
        # Remove Stack Overflow specific artifacts
        text = self.patterns['stackoverflow_artifacts'].sub('', text)
        
        # Remove user reputation patterns
        text = self.patterns['user_reputation'].sub('', text)
        text = self.patterns['username_reputation'].sub('', text)
        
        # Remove isolated numbers and vote counts
        text = self.patterns['isolated_numbers'].sub('', text)
        text = self.patterns['number_patterns'].sub(' ', text)
        
        # Remove timestamps and dates
        text = self.patterns['timestamps'].sub('', text)
        text = self.patterns['time_colons'].sub('', text)
        text = self.patterns['user_mentions'].sub('', text)
        
        # Remove specific Stack Overflow artifacts
        text = self.patterns['retryable_errors'].sub('', text)
        text = self.patterns['duplicate_questions'].sub('', text)
        text = self.patterns['answer_counts'].sub('', text)
        text = self.patterns['time_ago'].sub('', text)
        text = self.patterns['duplicate_markers'].sub('', text)
        
        # Remove common code artifacts and noise
        text = self.patterns['common_artifacts'].sub('', text)
        text = self.patterns['revision_thanks'].sub('', text)
        
        # Clean up extra whitespace
        text = self.patterns['multiple_newlines'].sub('\n\n', text)
        text = self.patterns['multiple_spaces'].sub(' ', text)
        text = text.strip()
        
        # Remove lines that are just numbers, symbols, or very short
        lines = text.split('\n')
        cleaned_lines = []
        for line in lines:
            line = line.strip()
            # Skip if line is empty, just numbers/symbols, or too short
            if (line and 
                len(line) > 3 and
                not self.patterns['meaningless_lines'].match(line) and
                not self.patterns['no_alphanumeric'].match(line)):
                cleaned_lines.append(line)
        
        return '\n'.join(cleaned_lines)
    
    def is_valid_content(self, text):
        """
        Check if the content is valid (not too short, contains meaningful text)
        """
        if not text or len(text.strip()) < 15:
            return False
        
        # Check if it's mostly just symbols or numbers
        alphanumeric = re.sub(r'[^a-zA-Z0-9]', '', text)
        if len(alphanumeric) < 10:
            return False
        
        # Check for meaningful words (at least 3 words with 3+ characters)
        words = self.patterns['meaningful_words'].findall(text)
        if len(words) < 3:
            return False
        
        return True
    
    def clean_and_convert_dataset(self, input_file, output_file):
        """
        Clean the messy dataset and convert to ChatML format
        """
        print(f"Loading dataset from {input_file}...")
        
        try:
            with open(input_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
        except Exception as e:
            print(f"Error loading dataset: {e}")
            return 0
        
        prompts = data.get("Prompt", [])
        instructions = data.get("Instruction", [])
        responses = data.get("Response", [])
        
        print(f"Found {len(prompts)} prompts, {len(instructions)} instructions, {len(responses)} responses")
        
        cleaned_conversations = []
        skipped = 0
        
        # Process each entry
        total_entries = max(len(prompts), len(instructions), len(responses))
        
        for i in range(total_entries):
            try:
                # Get raw content
                prompt = prompts[i] if i < len(prompts) else ""
                instruction = instructions[i] if i < len(instructions) else ""
                response = responses[i] if i < len(responses) else ""
                
                # Clean the content
                clean_prompt = self.clean_text(prompt)
                clean_instruction = self.clean_text(instruction)
                clean_response = self.clean_text(response)
                
                # Combine prompt and instruction for user message
                user_content = ""
                if clean_prompt and clean_instruction:
                    # If both exist, use prompt as title and instruction as details
                    user_content = f"{clean_prompt}\n\n{clean_instruction}"
                elif clean_prompt:
                    user_content = clean_prompt
                elif clean_instruction:
                    user_content = clean_instruction
                
                # Validate content
                if not self.is_valid_content(user_content) or not self.is_valid_content(clean_response):
                    skipped += 1
                    if i % 1000 == 0 and i > 0:
                        print(f"Processed {i}/{total_entries} entries, skipped {skipped} so far...")
                    continue
                
                # Create ChatML conversation
                conversation = {
                    "messages": [
                        {
                            "role": "user",
                            "content": user_content.strip()
                        },
                        {
                            "role": "assistant",
                            "content": clean_response.strip()
                        }
                    ]
                }
                
                cleaned_conversations.append(conversation)
                
                # Progress update
                if i % 1000 == 0 and i > 0:
                    print(f"Processed {i}/{total_entries} entries, {len(cleaned_conversations)} valid conversations...")
                
            except Exception as e:
                print(f"Error processing entry {i}: {e}")
                skipped += 1
                continue
        
        # Save to JSONL format
        print(f"Saving {len(cleaned_conversations)} conversations to {output_file}...")
        
        try:
            with open(output_file, 'w', encoding='utf-8') as file:
                for conversation in cleaned_conversations:
                    file.write(json.dumps(conversation, ensure_ascii=False) + '\n')
        except Exception as e:
            print(f"Error saving dataset: {e}")
            return 0
        
        print(f"✅ Successfully cleaned and converted {len(cleaned_conversations)} conversations")
        print(f"❌ Skipped {skipped} invalid entries")
        print(f"📁 Output saved to: {output_file}")
        
        return len(cleaned_conversations)
    
    def preview_cleaned_data(self, output_file, num_examples=3):
        """
        Preview the cleaned data
        """
        print(f"\n{'='*60}")
        print(f"PREVIEW OF CLEANED DATA")
        print(f"{'='*60}")
        
        try:
            with open(output_file, 'r', encoding='utf-8') as file:
                for i, line in enumerate(file):
                    if i >= num_examples:
                        break
                    
                    conversation = json.loads(line)
                    print(f"\n📝 CONVERSATION {i+1}:")
                    print(f"{'─'*40}")
                    
                    user_msg = conversation["messages"][0]["content"]
                    assistant_msg = conversation["messages"][1]["content"]
                    
                    print(f"👤 USER ({len(user_msg)} chars):")
                    print(user_msg[:300] + "..." if len(user_msg) > 300 else user_msg)
                    
                    print(f"\n🤖 ASSISTANT ({len(assistant_msg)} chars):")
                    print(assistant_msg[:300] + "..." if len(assistant_msg) > 300 else assistant_msg)
                    
                    print(f"{'─'*40}")
                    
        except Exception as e:
            print(f"Error previewing data: {e}")



In [9]:
def main():
    """
    Main function to run the dataset cleaning and conversion
    """
    print("🧹 DevOps Dataset Cleaner and ChatML Converter")
    print("=" * 50)
    
    cleaner = DatasetCleaner()
    
    # Clean and convert the dataset
    input_file = 'devops-42k.json'
    output_file = 'devops-42k-clean.jsonl'
    
    num_conversations = cleaner.clean_and_convert_dataset(input_file, output_file)
    
    # Preview the results if successful
    if num_conversations > 0:
        cleaner.preview_cleaned_data(output_file, num_examples=2)
        
        print(f"\n✨ Processing complete!")
        print(f"📊 Statistics:")
        print(f"   - Input file: {input_file}")
        print(f"   - Output file: {output_file}")
        print(f"   - Valid conversations: {num_conversations}")
        print(f"   - Ready for training! 🚀")
    else:
        print("❌ No valid conversations were generated. Check your input file.")

if __name__ == "__main__":
    main()

🧹 DevOps Dataset Cleaner and ChatML Converter
Loading dataset from devops-42k.json...
Found 42819 prompts, 42819 instructions, 42819 responses
Processed 1000/42819 entries, 998 valid conversations...
Processed 2000/42819 entries, 1998 valid conversations...
Processed 3000/42819 entries, 2997 valid conversations...
Processed 4000/42819 entries, 3995 valid conversations...
Processed 5000/42819 entries, 4993 valid conversations...
Processed 6000/42819 entries, 5989 valid conversations...
Processed 7000/42819 entries, 6988 valid conversations...
Processed 8000/42819 entries, 7987 valid conversations...
Processed 9000/42819 entries, 8985 valid conversations...
Processed 10000/42819 entries, 9983 valid conversations...
Processed 11000/42819 entries, 10981 valid conversations...
Processed 12000/42819 entries, 11981 valid conversations...
Processed 13000/42819 entries, 12981 valid conversations...
Processed 14000/42819 entries, 13979 valid conversations...
Processed 15000/42819 entries, 14975 